# Notebook 3: Approaches for Tabular Data

In this notebook we will show a few ways to use LLM pipelines with tabular data. I like to think of working with tabular data with LLMs with 3 options.

1. **Static Approach:** Store data in a knowledge base and try to retrieve the relevant data to answer the question directly
2. **Templated Code / Function Calling Approach:** Have pre-defined code that can run to satisfy certain requests. Extract inputs and produce answer using outputs with LLMs.
3. **AI Agent Approach:** The LLM creates code to run in order to produce analysis required to answer the question.
 
Complexity and reliance on the LLM increases from 1. where no code is executed to 2. where only pre-define coded is executed to 3. where the LLM produces code to execute. The separation is not perfectly clean agents can also just point only to pre-defined code, but this is just a framework. Additionally, for 1., the stored data could consist of extensively pre-process "views" of the data aimed at satisfying the majority of user queries. 

In this notebook, we will focus on 1. and 3., but demonstrate 2. through the use of agents to some extent. We will consider on Tesla (TSLA) stock prices data.
* First, we will use the common approach of using LangChain's [CSV Loader](https://python.langchain.com/docs/integrations/document_loaders/csv)
* Next, we will explore Langchain's [TimeWeightedVectorStoreRetriever](https://python.langchain.com/docs/modules/data_connection/retrievers/time_weighted_vectorstore) and learn why this may not be what we want
* Then, we will explore using agent based approaches from [LangChain](https://github.com/langchain-ai/langchain/tree/master) and [Pandas-AI](https://github.com/Sinaptik-AI/pandas-ai/tree/main)

## Import libraries and load the stock data

In [2]:
import pandas as pd
import os
import csv
import json
import time
import re
import numpy as np
from datetime import datetime

from sklearn.metrics.pairwise import cosine_similarity
import faiss

#We will use langchain to create a vector store to retrieve stronger negatives
from langchain.vectorstores.faiss import FAISS
from langchain.docstore import InMemoryDocstore
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.document_loaders import UnstructuredPDFLoader, csv_loader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.utils import mock_now


EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"## "BAAI/bge-base-en-v1.5""all-MiniLM-L6-v2"

In [33]:
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

True

In [3]:
#load stock data. Load dataframe and load directly as docs
tsla_stock = pd.read_csv("../data/TSLA.csv")
spy_etf = pd.read_csv("../data/SPY.csv") #for later

## Static Approaches 

Here we use a knowledge base of TSLA's stock data

#### Using LangChain's CSVLoader
This just loads each record as a json in an individual document in our vector database to retrieve

In [5]:
loader  = csv_loader.CSVLoader(file_path="../data/TSLA.csv")
stock_data_docs = loader.load()
stock_data_docs[0]

#for the retrieval process
embedding_function = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL_NAME,
        cache_folder="../models/sentencetransformers"
    )

In [4]:
#first let's add more context to help with Retrieval and add date to metadata (for later use)
for ii in range(0, len(stock_data_docs)):
    stock_data_docs[ii].page_content = "Daily stock market data for Tesla (TSLA):\n" + stock_data_docs[ii].page_content
    date = re.findall(r'Date: (\d{4}-\d{2}-\d{2})', stock_data_docs[ii].page_content)
    if len(date) > 0:
        stock_data_docs[ii].metadata['last_accessed_at'] = datetime.strptime(date[0], '%Y-%m-%d')
    else:
        stock_data_docs[ii].metadata['last_accessed_at'] = None
stock_data_docs[0]

Document(page_content='Daily stock market data for Tesla (TSLA):\nDate: 2010-06-29\nOpen: 1.266667\nHigh: 1.666667\nLow: 1.169333\nClose: 1.592667\nAdj Close: 1.592667\nVolume: 281494500', metadata={'source': '../data/TSLA.csv', 'row': 0, 'last_accessed_at': datetime.datetime(2010, 6, 29, 0, 0)})

In [5]:
db_data = FAISS.from_documents(stock_data_docs, embedding_function)
db_data.save_local("../data/faiss_stock")

Our stock data is just through 2024-02-02 so we will specify that as today's date

In [6]:
top_k=16
retriever_stock = VectorStoreRetriever(vectorstore=db_data, search_kwargs={"k": top_k})

def generate_response(prompt, retriever):
    #today's date - let's pretend it is 2024-02-02
    today = "2024-02-02"
    #replace "current" or "today" with today's date
    prompt = re.sub(r'current|today', today, prompt, flags=re.IGNORECASE)
    print("Prompt: ", prompt)
    # Get the top k most similar documents
    results = retriever.get_relevant_documents(prompt)
    return results

In [7]:
question ="What is TSLA's current close price?"
generate_response(question, retriever_stock)

Prompt:  What is TSLA's 2024-02-02 close price?


[Document(page_content='Daily stock market data for Tesla (TSLA):\nDate: 2021-08-03\nOpen: 239.666672\nHigh: 240.883331\nLow: 233.669998\nClose: 236.580002\nAdj Close: 236.580002\nVolume: 64860900', metadata={'source': '../data/TSLA.csv', 'row': 2793, 'last_accessed_at': datetime.datetime(2021, 8, 3, 0, 0)}),
 Document(page_content='Daily stock market data for Tesla (TSLA):\nDate: 2021-04-20\nOpen: 239.139999\nHigh: 245.750000\nLow: 236.896667\nClose: 239.663330\nAdj Close: 239.663330\nVolume: 106827000', metadata={'source': '../data/TSLA.csv', 'row': 2720, 'last_accessed_at': datetime.datetime(2021, 4, 20, 0, 0)}),
 Document(page_content='Daily stock market data for Tesla (TSLA):\nDate: 2022-08-18\nOpen: 306.000000\nHigh: 306.500000\nLow: 301.853333\nClose: 302.869995\nAdj Close: 302.869995\nVolume: 47500500', metadata={'source': '../data/TSLA.csv', 'row': 3056, 'last_accessed_at': datetime.datetime(2022, 8, 18, 0, 0)}),
 Document(page_content='Daily stock market data for Tesla (TSLA)

Notice that even when we replace "current" with today's date, our retrieval process is not strong enough to only pick recent dates. 

Let's try using TimeWeightedVectorStoreRetriever to bias towards recent dates...<br>

#### TimeWeightedVectorStoreRetriever

In [8]:
# Solution 1
embedding_size = len(embedding_function.embed_documents([question])[0])
index = faiss.IndexFlatL2(384)
vectorstore = FAISS(embedding_function, index, InMemoryDocstore({}), {})
tw_retriever_stock = TimeWeightedVectorStoreRetriever(vectorstore=vectorstore,decay_rate=0.005, k=top_k)
# Notice the last access time is that date time
tw_retriever_stock.add_documents(stock_data_docs)
with mock_now(datetime(2024, 2, 2, 23, 0)):
    rel_docs = generate_response(question, tw_retriever_stock)
rel_docs

Prompt:  What is TSLA's 2024-02-02 close price?


[Document(page_content='Daily stock market data for Tesla (TSLA):\nDate: 2024-02-02\nOpen: 185.039993\nHigh: 188.690002\nLow: 182.000000\nClose: 187.910004\nAdj Close: 187.910004\nVolume: 110505100', metadata={'source': '../data/TSLA.csv', 'row': 3422, 'last_accessed_at': MockDateTime(2024, 2, 2, 23, 0), 'created_at': datetime.datetime(2024, 3, 27, 10, 23, 4, 663286), 'buffer_idx': 3422}),
 Document(page_content='Daily stock market data for Tesla (TSLA):\nDate: 2024-02-01\nOpen: 188.500000\nHigh: 189.880005\nLow: 184.279999\nClose: 188.860001\nAdj Close: 188.860001\nVolume: 91843300', metadata={'source': '../data/TSLA.csv', 'row': 3421, 'last_accessed_at': MockDateTime(2024, 2, 2, 23, 0), 'created_at': datetime.datetime(2024, 3, 27, 10, 23, 4, 663286), 'buffer_idx': 3421}),
 Document(page_content='Daily stock market data for Tesla (TSLA):\nDate: 2024-01-31\nOpen: 187.000000\nHigh: 193.970001\nLow: 185.850006\nClose: 187.289993\nAdj Close: 187.289993\nVolume: 103221400', metadata={'sour

This technically worked, but is not what we want. The 'last_accessed_at' was updated so it is not longer using the date for the stock and this will hurt performance when asking for previous dates. Let's try agents.

## Agent-based Approaches

#### Langchain CSV / Pandas Agents

Here we will show simple use of Langchain [csv](https://python.langchain.com/docs/integrations/toolkits/csv) and [Pandas](https://python.langchain.com/docs/integrations/toolkits/pandas) agents. The csv agent uses the Pandas agent under the hood so we will just use the pandas agent directly. 

In [6]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

True

In [11]:
tsla_stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,1.592667,281494500
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500


In [18]:
def try_agent(agent, prompt, verbose=True):
    response = None
    try:
        response = agent.invoke(prompt, verbose=verbose)
        print(response)
    except Exception as e:
        print(e)
    return response

#### First we'll try the ZERO_SHOT_REACT_DESCRIPTION agent

**Warning:** this does not work well

**Also, it works much better with Langchain's [ChatOpenAI](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/chat_models/openai.py#L351) that just Langchain's [OpenAI](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/llms/openai.py#L731) which is used in the zero shot example**

In [238]:
llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")
# llm = OpenAI(temperature=0, model="gpt-4-1106-preview")
agent = create_pandas_dataframe_agent(llm, tsla_stock, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [239]:
response = try_agent(agent, "What was the most recent close price for TSLA?", verbose=True)



> Entering new AgentExecutor chain...
Thought: To find the most recent close price for TSLA, I need to look at the last row of the dataframe and get the value from the 'Close' column.

Action: python_repl_ast
Action Input: df.tail(1)['Close'].iloc[0]187.910004I now know the final answer.

Final Answer: The most recent close price for TSLA was $187.91.

> Finished chain.
{'input': 'What was the most recent close price for TSLA?', 'output': 'The most recent close price for TSLA was $187.91.'}


In [240]:
response = try_agent(agent, "Does the change in open or close prices have a larger standard deviation?", verbose=True)



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to calculate the standard deviation of the changes in open prices and the changes in close prices. The change can be calculated by subtracting the previous day's price from the current day's price. I will use the `diff()` method to calculate the changes and then the `std()` method to calculate the standard deviation for both 'Open' and 'Close' columns.

Action: Calculate the standard deviation of the changes in open prices.
Action Input: df['Open'].diff().std()
Calculate the standard deviation of the changes in open prices. is not a valid tool, try one of [python_repl_ast].I need to use the python_repl_ast tool to execute the Python command.

Action: python_repl_ast
Action Input: df['Open'].diff().std()4.945367157862929The standard deviation of the changes in open prices is approximately 4.945.

Action: Calculate the standard deviation of the changes in close prices.
Action Input: df['Close'].diff().std()C

That sort-of worked when I ran it (sometimes errors). Sometimes we can see that it enters another step. 

 We can look at [Langchain's default code](https://github.com/langchain-ai/langchain/blob/master/libs/experimental/langchain_experimental/agents/agent_toolkits/pandas/base.py) and [prompts](https://github.com/langchain-ai/langchain/blob/master/libs/experimental/langchain_experimental/agents/agent_toolkits/pandas/prompt.py) to see how we can improve it and work around bugs.

The issue is in part related to a bug in Langchain. Let's use a work around and define out own prompt (which we will see another very minor bug).

- **Bug 1:** The stop token in the React Agent is \nObservation, but needs to be \nFinal Answer for Pandas agent
- **Bug 2:** "create_pandas_dataframe_agent" requiring include_df_in_prompt to be None vs False.


Building our own prompts for the agent:

In [241]:
PREFIX = """
You are a large language model being used in an agent workflow involving questions, thoughts, use of tools, and actions using those tools. One of those tools allows you to work with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:\n"""

In [242]:
tsla_stock.describe(include='all')
#add first 2 rows and last 2 rows to describe
df_describe = tsla_stock.describe(include='all')
df_describe.loc['1st_row'] = tsla_stock.iloc[0]
df_describe.loc['2nd_row'] = tsla_stock.iloc[1]
df_describe.loc['2nd_last_row'] = tsla_stock.iloc[-2]
df_describe.loc['last_row'] = tsla_stock.iloc[-1]
df_describe

,Date,Open,High,Low,Close,Adj Close,Volume
count,3423,3423.000000,3423.000000,3423.000000,3423.000000,3423.000000,3.423000e+03
unique,3423,NaN,NaN,NaN,NaN,NaN,NaN
top,2010-06-29,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,71.463546,73.027288,69.772792,71.436731,71.436731,9.690499e+07
std,NaN,101.915934,104.171288,99.425029,101.828184,101.828184,7.980646e+07
min,NaN,1.076000,1.108667,0.998667,1.053333,1.053333,1.777500e+06
25%,NaN,10.870667,11.166334,10.688667,10.945000,10.945000,4.623000e+07
50%,NaN,17.025333,17.266666,16.719334,16.999332,16.999332,8.160300e+07
75%,NaN,122.324997,126.674999,119.958332,123.389999,123.389999,1.232748e+08


In [243]:
SUFFIX_WITH_DF_DESCRIBE = """
You must STOP once you found the final answer. This is the result of running:

df_describe = df.describe(include='all')
df_describe.loc['1st_row'] = df.iloc[0]
df_describe.loc['2nd_row'] = df.iloc[1]
df_describe.loc['2nd_last_row'] = df.iloc[-2]
df_describe.loc['last_row'] = df.iloc[-1]
print(df_describe.to_markdown())
del df_describe:
{df_describe}

Please note that python_repl_ast cannot use df_decribe, but can use df.

Begin!
Question: {{input}}
{{agent_scratchpad}}"""

In [244]:
df_describe_str = str(df_describe.to_markdown())#json.dumps((df_describe.to_json())).replace("{", "{{").replace("}", "}}")
partial_format = SUFFIX_WITH_DF_DESCRIBE.format(df_describe=df_describe_str)
SUFFIX_FINAL = partial_format.replace("{{input}}", "{input}").replace("{{agent_scratchpad}}", "{agent_scratchpad}")
print(SUFFIX_FINAL)


You must STOP once you found the final answer. This is the result of running:

df_describe = df.describe(include='all')
df_describe.loc['1st_row'] = df.iloc[0]
df_describe.loc['2nd_row'] = df.iloc[1]
df_describe.loc['2nd_last_row'] = df.iloc[-2]
df_describe.loc['last_row'] = df.iloc[-1]
print(df_describe.to_markdown())
del df_describe:
|              | Date       |       Open |       High |         Low |      Close |   Adj Close |         Volume |
|:-------------|:-----------|-----------:|-----------:|------------:|-----------:|------------:|---------------:|
| count        | 3423       | 3423       | 3423       | 3423        | 3423       |  3423       | 3423           |
| unique       | 3423       |  nan       |  nan       |  nan        |  nan       |   nan       |  nan           |
| top          | 2010-06-29 |  nan       |  nan       |  nan        |  nan       |   nan       |  nan           |
| freq         | 1          |  nan       |  nan       |  nan        |  nan       |   nan   

Now, let's use a workaround for the Langchain bug:

In [245]:
llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")

#Bug 1 workaround: the stop token in the React Agent is \nObservation, but needs to be \nFinal Answer for Pandas agent
llm_with_stop = llm.bind(stop=["\nFinal Answer"])

In [246]:
#Bug 2 workaround:
agent = create_pandas_dataframe_agent(llm_with_stop, tsla_stock, include_df_in_prompt = None, #handle_parsing_errors=True,
                                      agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, prefix=PREFIX, suffix=SUFFIX_FINAL, verbose=True)

In [247]:
response = try_agent(agent, "What was the most recent close price for TSLA?", verbose=True)



> Entering new AgentExecutor chain...
Thought: The most recent close price for TSLA can be found in the 'Close' column of the last row of the dataframe `df`.

Action: python_repl_ast
Action Input: df['Close'].iloc[-1]187.910004I now know the final answer.
Final Answer: The most recent close price for TSLA was $187.91.

> Finished chain.
{'input': 'What was the most recent close price for TSLA?', 'output': 'The most recent close price for TSLA was $187.91.'}


In [248]:
response = try_agent(agent, "Does the change in open or close prices have a larger standard deviation?", verbose=True)



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to compare the standard deviation of the 'Open' prices with the standard deviation of the 'Close' prices.

Action: python_repl_ast
Action Input: df['Open'].std(), df['Close'].std()(101.91593442111841, 101.82818449060503)I now know the final answer
Final Answer: The 'Open' prices have a slightly larger standard deviation than the 'Close' prices.

> Finished chain.
{'input': 'Does the change in open or close prices have a larger standard deviation?', 'output': "The 'Open' prices have a slightly larger standard deviation than the 'Close' prices."}


That failed to take the difference and I had to significantly change the PREFIX and SUFFIX to get it to sometimes work. We can take a look at the Langchain code again, but let's move on.

#### Let's now try the OPENAI_FUNCTIONS agent in Langchain

**Note:** This works very badly with "gpt-4-1106-preview", but a bit better with "gpt-4" and "gpt-3.5-turbo-0613"

In [258]:
llm = ChatOpenAI(temperature=0, model="gpt-4")
agent = create_pandas_dataframe_agent(llm, tsla_stock,verbose=True,agent_type=AgentType.OPENAI_FUNCTIONS)

In [259]:
response = try_agent(agent, "What was the most recent close price for TSLA?", verbose=True)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Close'].iloc[-1]"}`


187.910004The most recent close price for TSLA was $187.91.

> Finished chain.
{'input': 'What was the most recent close price for TSLA?', 'output': 'The most recent close price for TSLA was $187.91.'}


In [260]:
response = try_agent(agent, "Does the change in open or close prices have a larger standard deviation?", verbose=True)



> Entering new AgentExecutor chain...



Invoking: `python_repl_ast` with `{'query': "df['Open'].diff().std(), df['Close'].diff().std()"}`


(4.945367157862929, 4.678933840976249)The standard deviation of the change in open prices is larger than the standard deviation of the change in close prices.

> Finished chain.
{'input': 'Does the change in open or close prices have a larger standard deviation?', 'output': 'The standard deviation of the change in open prices is larger than the standard deviation of the change in close prices.'}


Let's forget about Langchain Agents for now as it is a little bit of a pain ... 

### Let's try Agents with [Pandas AI](https://docs.pandas-ai.com/en/latest/)

In [2]:
import pandas as pd
from pandasai import SmartDatalake
from pandasai import SmartDataframe
from pandasai.llm import OpenAI
from pandasai import Agent
from pandasai.skills import skill

In [8]:
llm = OpenAI(model="gpt-4-1106-preview", verbose=True)

Let's first just use the simple agent with SmartDataframe

In [15]:
agent = SmartDataframe(tsla_stock, description ="TSLA Stock Market Data", config={"llm": llm})
agent.chat("Does the change in open or close prices have a larger standard deviation?")

'Close prices have a larger standard deviation.'

That actually did not work at all - looking at the log below shows it just made up the answer.

Let's show something a bit more interesting using the Agent framework from Pandas-AI which allows us to directly inform our LLM or "skills" we added

In [34]:
# Function doc string to give more context to the model for use this skill
@skill
def get_capm_params(stock_data: pd.DataFrame, benchmark_data: pd.DataFrame):
    """
    This function estimates an Intercept(alpha) and a beta by regressing the stock returns on the benchmark returns.
    This is the Capital Asset Pricing Model (CAPM) model.
    Args:
        stock_data: A Pandas DataFrame with stock data
        benchmark_data: A Pandas DataFrame with the benchmark's data
    Returns:
        alpha: Intercept
        beta: Beta
    """
    from sklearn.linear_model import LinearRegression
    import pandas as pd
    #calculate stock returns
    stock_data['Return'] = stock_data['Close'].pct_change()
    stock_data.dropna(inplace=True)
    #calculate benchmark returns
    benchmark_data['Return'] = benchmark_data['Close'].pct_change()
    benchmark_data.dropna(inplace=True)
    #perform inner join to get common dates
    stock_data = stock_data.merge(benchmark_data, on='Date', how='inner', suffixes=('_stock', '_benchmark'))
    X = stock_data[['Return_benchmark']]
    y = stock_data['Return_stock']
    model = LinearRegression().fit(X, y)
    alpha = model.intercept_
    beta = model.coef_[0]
    return alpha, beta

agent = Agent([tsla_stock, spy_etf], config={"llm": llm}, memory_size=10, description="A list of 2 DataFrames: TSLA stock data and SPY ETF data respectively.")

agent.add_skills(get_capm_params)

In [44]:
get_capm_params(tsla_stock, spy_etf)

(0.0013747603371657858, 1.4142763404820184)

In [36]:
# Chat with the agent
response = agent.chat("What is the Beta of TSLA? You can use the SPY ETF as the benchmark and use the CAPM model.")
print(response)

1.4098669332856566


We got the right answer, but the beta is slightly different - let's check the logs ... 

This appears to just be estimation sensitivity so all good!

In [37]:
#print pandasai.log file to see the logs
# with open("pandasai.log", "r") as f:
#     print(f.read())

This seems promising! So far we have just shown pretty simple agents and can satisfy requirements as long as the users can read the code and understand all he intermediate steps.


#### SQL Agent - WORK IN PROGRESS

In [7]:
import sqlite3
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, Float, String, DateTime


In [16]:
engine = create_engine('sqlite:///tsla_stock.db')
connection = engine.connect()

In [11]:
tsla_stock['Symbol'] = 'TSLA'
tsla_stock.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2010-06-29,1.266667,1.666667,1.169333,1.592667,1.592667,281494500,TSLA
1,2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500,TSLA
2,2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000,TSLA
3,2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000,TSLA
4,2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500,TSLA


In [12]:
# Using SQLAlchemy to define a table schema
metadata = MetaData()

stocks = Table('stocks', metadata,
    Column('Symbol', String, comment='Stock symbol'),
    Column('Date', DateTime, comment='Date of trading'),
    Column('Open', Float, comment='Opening price'),
    Column('High', Float, comment='Highest price during the trading day'),
    Column('Low', Float, comment='Lowest price during the trading day'),
    Column('Close', Float, comment='Closing price at the end of trading day'),
    Column('Adj Close', Float, comment='Adjusted closing price'),
    Column('Volume', Integer, comment='Number of shares traded'),
)
metadata.create_all(engine)  # Creates the table

In [13]:
tsla_stock.to_sql('stocks', con=engine, if_exists='replace', index=False)

3423

In [25]:
### In sqlite3 comments are not accessible so we will add a table to store column descriptions
descriptions = {
    'Column': ['Symbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'],
    'Description': [
        'Stock symbol', 'Date of trading', 'Opening price', 'Highest price during the trading day',
        'Lowest price during the trading day', 'Closing price at the end of trading day',
        'Adjusted closing price', 'Number of shares traded'
    ]
}
desc_df = pd.DataFrame(descriptions)
desc_df['Table'] = 'stocks'
desc_df[['Table','Column','Description']].to_sql('column_descriptions', con=engine, if_exists='replace', index=False)

8

In [26]:
# Create a connection to the SQLite database
conn = sqlite3.connect('tsla_stock.db')

# Define your SQL query
query = "SELECT * FROM stocks"

# Use pandas to execute the query and load the result into a DataFrame
try:
    df = pd.read_sql_query(query, conn)
    print(df.head(2))
    descriptions = pd.read_sql_query("SELECT * FROM column_descriptions", conn)
    print(descriptions)
except Exception as e:
    print("An error occurred:", e)

# Close the connection
conn.close()



         Date      Open      High       Low     Close  Adj Close     Volume  \
0  2010-06-29  1.266667  1.666667  1.169333  1.592667   1.592667  281494500   
1  2010-06-30  1.719333  2.028000  1.553333  1.588667   1.588667  257806500   

  Symbol  
0   TSLA  
1   TSLA  
    Table     Column                              Description
0  stocks     Symbol                             Stock symbol
1  stocks       Date                          Date of trading
2  stocks       Open                            Opening price
3  stocks       High     Highest price during the trading day
4  stocks        Low      Lowest price during the trading day
5  stocks      Close  Closing price at the end of trading day
6  stocks  Adj Close                   Adjusted closing price
7  stocks     Volume                  Number of shares traded


In [24]:
SQL_SYSTEM_PROMPT = """"
You are a sophisticated generative AI trained to understand human language queries about stock market data and translate them into SQL queries. The database you are querying contains a table named stocks with the following columns:

Symbol: The stock symbol.
Date: The date of the trading activity.
Open: The opening price of the stock on that day.
High: The highest price of the stock during the trading day.
Low: The lowest price of the stock during the trading day.
Close: The closing price of the stock at the end of the trading day.
Adj Close: The adjusted closing price after accounting for any corporate actions.
Volume: The number of shares traded during the day.
Tasks:

Convert the human language query into a precise SQL query.
Ensure to accurately interpret the key details from the human query such as specific dates, stock symbols, price ranges, and other relevant filters.
Your responses should strictly be SQL queries; do not execute them.
Examples:

Human Query: "Show me all transactions of AAPL stocks from January 2021."
AI Response:
```sql
SELECT * FROM stocks WHERE Symbol = 'AAPL' AND Date >= '2021-01-01' AND Date < '2022-01-01';
```
Human Query: "What was the highest price of MSFT in the last year?"
AI Response:
```sql
SELECT MAX(High) FROM stocks WHERE Symbol = 'MSFT' AND Date >= date('now', '-1 year');
```
Human Query: "List all closing prices below $50 for any stock last month."
AI Response:
```sql
Copy code
SELECT Symbol, Date, Close FROM stocks WHERE Close < 50 AND Date >= date('now', 'start of month', '-1 month') AND Date < date('now', 'start of month');
```
Instructions:

Always validate the input query to ensure clarity and completeness.
Extract relevant temporal and financial parameters from the query.
Construct SQL queries that are syntactically correct and formatted for readability.
Assume the queries are to be executed in a SQLite database environment.
"""

In [30]:
descriptions.to_markdown(index=False)

'| Table   | Column    | Description                             |\n|:--------|:----------|:----------------------------------------|\n| stocks  | Symbol    | Stock symbol                            |\n| stocks  | Date      | Date of trading                         |\n| stocks  | Open      | Opening price                           |\n| stocks  | High      | Highest price during the trading day    |\n| stocks  | Low       | Lowest price during the trading day     |\n| stocks  | Close     | Closing price at the end of trading day |\n| stocks  | Adj Close | Adjusted closing price                  |\n| stocks  | Volume    | Number of shares traded                 |'

In [53]:
SQL_SYSTEM_PROMPT2 = """
You are a sophisticated generative AI model designed to convert human language queries into SQL queries. Your operation is based on a dynamic understanding of database schemas through a descriptions table. This descriptions table stores metadata about tables and columns in the database and includes the following columns:

Table: The name of the table to which the description relates.
Column: The actual column name in the table.
Description: A human-readable description of what the column represents.

**Example Description Table:**
| Table   | Column    | Description                             |
|:--------|:----------|:----------------------------------------|
| stocks  | Symbol    | Stock symbol                            |
| stocks  | Date      | Date of trading                         |
| stocks  | Open      | Opening price                           |
| stocks  | High      | Highest price during the trading day    |
| stocks  | Low       | Lowest price during the trading day     |
| stocks  | Close     | Closing price at the end of trading day |
| stocks  | Adj Close | Adjusted closing price                  |
| stocks  | Volume    | Number of shares traded                 |

Tasks:
Convert the human language query into a precise SQL query.
Ensure to accurately interpret the key details from the human query such as specific dates and other relevant filters.
Your responses should strictly be SQL queries; do not execute them.

SQL Response examples using the example descriptions table:
Human Query: "Show me all transactions of AAPL stocks from January 2021."
AI Response:
```sql
SELECT * FROM stocks WHERE Symbol = 'AAPL' AND Date >= '2021-01-01' AND Date < '2022-01-01';
```
Human Query: "What was the highest price of MSFT in the last year?"
AI Response:
```sql
SELECT MAX(High) FROM stocks WHERE Symbol = 'MSFT' AND Date >= date('now', '-1 year');
```
Human Query: "List all closing prices below $50 for any stock last month."
AI Response:
```sql
SELECT Symbol, Date, Close FROM stocks WHERE Close < 50 AND Date >= date('now', 'start of month', '-1 month') AND Date < date('now', 'start of month');
```

Now it's your turn to generate SQL queries based on the provided human language queries.

Description Table:
{descriptions_table}

Instructions:

Utilize the descriptions in the descriptions table to map human-language queries to appropriate SQL queries.
Pay careful attention to interpreting the temporal parameters specified in queries.
Generate SQL queries that are syntactically correct, ensuring they reflect the semantic understanding of the database structure based on descriptions.
Operate under the assumption that the database environment supports typical SQL functions and date calculations.
Remember to use quotes for column names that include a space when generating SQL queries.
"""

SQL_SYSTEM_PROMPT2 = SQL_SYSTEM_PROMPT2.format(descriptions_table=descriptions.to_markdown(index=False))
print(SQL_SYSTEM_PROMPT2)


You are a sophisticated generative AI model designed to convert human language queries into SQL queries. Your operation is based on a dynamic understanding of database schemas through a descriptions table. This descriptions table stores metadata about tables and columns in the database and includes the following columns:

Table: The name of the table to which the description relates.
Column: The actual column name in the table.
Description: A human-readable description of what the column represents.

**Example Description Table:**
| Table   | Column    | Description                             |
|:--------|:----------|:----------------------------------------|
| stocks  | Symbol    | Stock symbol                            |
| stocks  | Date      | Date of trading                         |
| stocks  | Open      | Opening price                           |
| stocks  | High      | Highest price during the trading day    |
| stocks  | Low       | Lowest price during the trading day     |


In [ ]:
import openai
openai.api_key = os.getenv('OPENAI_API_KEY')

In [54]:
response_full = openai.chat.completions.create(model="gpt-4o", messages=[{"role": "system", "content": SQL_SYSTEM_PROMPT2},
                                                                      {"role": "user", "content": "What was the highest price of TSLA in the last year?" }],
                                                                      temperature=0) #, logprobs=True, top_logprobs=5

print(response_full.choices[0].message.content)

#extract the SQL query from the response
sql_query = response_full.choices[0].message.content.split("```sql")[1].split("```")[0].strip()

#run the sql query using the conn connection 
conn = sqlite3.connect('tsla_stock.db')
cursor = conn.cursor()
cursor.execute(sql_query)

# Get column names from cursor.description
column_names = [description[0] for description in cursor.description]
result = cursor.fetchall()
result_with_column_names = [column_names] + result

print(result_with_column_names)

```sql
SELECT MAX(High) FROM stocks WHERE Symbol = 'TSLA' AND Date >= date('now', '-1 year');
```
[['MAX(High)'], (299.290009,)]


In [55]:
response_full = openai.chat.completions.create(model="gpt-4o", messages=[{"role": "system", "content": SQL_SYSTEM_PROMPT2},
                                                                      {"role": "user", "content": "What was the average daily high price of TSLA in December 2023?" }],
                                                                      temperature=0) #, logprobs=True, top_logprobs=5

print(response_full.choices[0].message.content)

#extract the SQL query from the response
sql_query = response_full.choices[0].message.content.split("```sql")[1].split("```")[0].strip()

#run the sql query using the conn connection 
conn = sqlite3.connect('tsla_stock.db')
cursor = conn.cursor()
cursor.execute(sql_query)

# Get column names from cursor.description
column_names = [description[0] for description in cursor.description]
result = cursor.fetchall()
result_with_column_names = [column_names] + result

print(result_with_column_names)

```sql
SELECT AVG(High) FROM stocks WHERE Symbol = 'TSLA' AND Date >= '2023-12-01' AND Date < '2024-01-01';
```
[['AVG(High)'], (251.22250119999998,)]


In [56]:
response_full = openai.chat.completions.create(model="gpt-4o", messages=[{"role": "system", "content": SQL_SYSTEM_PROMPT2},
                                                                      {"role": "user", "content": "Show me all trade data for TSLA stocks from January 2024." }],
                                                                      temperature=0) #, logprobs=True, top_logprobs=5

print(response_full.choices[0].message.content)

#extract the SQL query from the response
sql_query = response_full.choices[0].message.content.split("```sql")[1].split("```")[0].strip()

#run the sql query using the conn connection 
conn = sqlite3.connect('tsla_stock.db')
cursor = conn.cursor()
cursor.execute(sql_query)

# Get column names from cursor.description
column_names = [description[0] for description in cursor.description]
result = cursor.fetchall()
result_with_column_names = [column_names] + result
print(result_with_column_names)



```sql
SELECT * FROM stocks WHERE Symbol = 'TSLA' AND Date >= '2024-01-01' AND Date < '2024-02-01';
```
[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol'], ('2024-01-02', 250.080002, 251.25, 244.410004, 248.419998, 248.419998, 104654200, 'TSLA'), ('2024-01-03', 244.979996, 245.679993, 236.320007, 238.449997, 238.449997, 121082600, 'TSLA'), ('2024-01-04', 239.25, 242.699997, 237.729996, 237.929993, 237.929993, 102629300, 'TSLA'), ('2024-01-05', 236.860001, 240.119995, 234.899994, 237.490005, 237.490005, 92379400, 'TSLA'), ('2024-01-08', 236.139999, 241.25, 235.300003, 240.449997, 240.449997, 85166600, 'TSLA'), ('2024-01-09', 238.110001, 238.960007, 232.039993, 234.960007, 234.960007, 96705700, 'TSLA'), ('2024-01-10', 235.100006, 235.5, 231.289993, 233.940002, 233.940002, 91628500, 'TSLA'), ('2024-01-11', 230.570007, 230.929993, 225.369995, 227.220001, 227.220001, 105873600, 'TSLA'), ('2024-01-12', 220.080002, 225.339996, 217.149994, 218.889999, 218.889999, 122889

In [57]:
df = pd.DataFrame(result, columns=column_names)
df

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2024-01-02,250.080002,251.250000,244.410004,248.419998,248.419998,104654200,TSLA
1,2024-01-03,244.979996,245.679993,236.320007,238.449997,238.449997,121082600,TSLA
2,2024-01-04,239.250000,242.699997,237.729996,237.929993,237.929993,102629300,TSLA
3,2024-01-05,236.860001,240.119995,234.899994,237.490005,237.490005,92379400,TSLA
4,2024-01-08,236.139999,241.250000,235.300003,240.449997,240.449997,85166600,TSLA
5,2024-01-09,238.110001,238.960007,232.039993,234.960007,234.960007,96705700,TSLA
6,2024-01-10,235.100006,235.500000,231.289993,233.940002,233.940002,91628500,TSLA
7,2024-01-11,230.570007,230.929993,225.369995,227.220001,227.220001,105873600,TSLA
8,2024-01-12,220.080002,225.339996,217.149994,218.889999,218.889999,122889000,TSLA
9,2024-01-16,215.100006,223.490005,212.179993,219.910004,219.910004,115355000,TSLA


In [58]:
response_full = openai.chat.completions.create(model="gpt-4o", messages=[{"role": "system", "content": SQL_SYSTEM_PROMPT2},
                                                                      {"role": "user", "content": "Create a trend plot for TSLA's prices over the last year" }],
                                                                      temperature=0) #, logprobs=True, top_logprobs=5

print(response_full.choices[0].message.content)

#extract the SQL query from the response
sql_query = response_full.choices[0].message.content.split("```sql")[1].split("```")[0].strip()

#run the sql query using the conn connection 
conn = sqlite3.connect('tsla_stock.db')
cursor = conn.cursor()
cursor.execute(sql_query)

# Get column names from cursor.description
column_names = [description[0] for description in cursor.description]
result = cursor.fetchall()
result_with_column_names = [column_names] + result
print(result_with_column_names)

```sql
SELECT Date, Open, High, Low, Close, "Adj Close"
FROM stocks
WHERE Symbol = 'TSLA' AND Date >= date('now', '-1 year')
ORDER BY Date;
```
[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close'], ('2023-05-15', 167.660004, 169.759995, 164.550003, 166.350006, 166.350006), ('2023-05-16', 165.649994, 169.520004, 164.350006, 166.520004, 166.520004), ('2023-05-17', 168.410004, 174.5, 167.190002, 173.860001, 173.860001), ('2023-05-18', 174.220001, 177.059998, 172.449997, 176.889999, 176.889999), ('2023-05-19', 177.169998, 181.949997, 176.309998, 180.139999, 180.139999), ('2023-05-22', 180.699997, 189.320007, 180.110001, 188.869995, 188.869995), ('2023-05-23', 186.199997, 192.960007, 185.259995, 185.770004, 185.770004), ('2023-05-24', 182.229996, 184.220001, 178.220001, 182.899994, 182.899994), ('2023-05-25', 186.539993, 186.779999, 180.580002, 184.470001, 184.470001), ('2023-05-26', 184.619995, 198.600006, 184.529999, 193.169998, 193.169998), ('2023-05-30', 200.100006, 204.479996, 197.52

Based on trying a few other prompts and tasks as well as diagnosing all of the above to make it work, I believe a more robust approach is needed to make this work for an end-user that might not read code or understand intermediate steps. This may be possible in LangChain or with Pandas-AI, but there are also building from scratch or using an existing full-scale agent-based frameworks such as [AutoGen](https://github.com/microsoft/autogen), [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT), [OpenAgents](https://github.com/Significant-Gravitas/AutoGPT) and [crewAI](https://github.com/joaomdmoura/crewAI)